<a href="https://colab.research.google.com/github/rselent/DS-Unit-4-Sprint-3-Deep-Learning/blob/master/module1-rnn-and-lstm/4_3_1-RNN_and_LSTM-Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [0]:
!pip install requests --upgrade
!pip install tensorflow --upgrade

In [0]:
# Words, words, mere words, no matter from the heart.

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

import numpy as np
import requests as rq
import random
import sys
import os

In [0]:
def sample( preds):
    # sample index from probability array
    preds = np.asarray( preds).astype( 'float64')
    preds = np.log( preds) / 1
    expreds = np.exp( preds)
    preds = expreds / np.sum( expreds)
    probas = np.random.multinomial( 1, preds, 1)
    return np.argmax( probas)

def endofEpoch( epoch, _):
    # print generated text
    
    print()
    print( '----- Generating text after Epoch: {}'.format( epoch))
    
    startIndex = random.randint( 0, len( text) - maxLength - 1)
    
    generated = ''
    
    sentence = text[ startIndex: startIndex + maxLength]
    generated += sentence
    
    print( '----- Generating with seed: "' + sentence + '"')
    sys.stdout.write( generated)
    
    for i in range( 400):
        xPred = np.zeros( (1, maxLength, len( chars)))
        for t, char in enumerate( sentence):
            xPred[ 0, t, chartoInt[ char]] = 1
            
        preds = model.predict( xPred, verbose= 0)[0]
        nextIndex = sample( preds)
        nextChar = inttoChar[ nextIndex]
        
        sentence = sentence[ 1:] + nextChar
        
        sys.stdout.write( nextChar)
        sys.stdout.flush()
    print()

printCallback = LambdaCallback( on_epoch_end= endofEpoch)

In [4]:
# My words fly up, my thoughts remain below: words without thoughts never to heaven go.

raw = rq.get( 'https://www.gutenberg.org/files/100/100-0.txt')
raw.encoding = 'utf-8'

text = raw.text

# check length of document... should be 5mil+
len( text)

5740054

In [5]:
## FOR TESTING ##
text = text[1000000:1500000]
len( text)

500000

In [6]:
# reduce to unique chars
chars = list( set( text))

# lookup tables
chartoInt = { c:i for i, c in enumerate( chars)}
inttoChar = { i:c for i, c in enumerate( chars)}

len( chars)

80

In [18]:
# Tomorrow, and tomorrow, and tomorrow...

maxLength = 32
step = 5

encodededed = [chartoInt[ c] for c in text]

sequences = []      # each element = 40char long
nextChar = []       # one element for each sequence

for i in range( 0, len( encodededed) - maxLength, step):
    sequences.append( encodededed[ i : i + maxLength])
    nextChar.append( encodededed[ i + maxLength])
print( "sequences: ", len( sequences))

sequences:  99994


In [19]:
x = np.zeros(( len( sequences), maxLength, len( chars)), dtype= np.bool)
y = np.zeros(( len( sequences), len( chars)), dtype= np.bool)

for i, sequence in enumerate( sequences):
    for t, char in enumerate( sequence):
        x[ i, t, char] = 1
    y[ i, nextChar[i]] = 1

x.shape, y.shape

((99994, 32, 80), (99994, 80))

In [0]:
model = Sequential()
model.add( LSTM( 256, input_shape= (maxLength, len( chars))))
model.add( Dense( len( chars), activation= 'softmax'))

model.compile( loss= 'categorical_crossentropy', optimizer= 'adam')

In [21]:
# OUT, OUT brief candle...

model.fit( x, y,
           batch_size= 64,
           epochs= 8,
           callbacks= [print_callback]
)

10560/99994 [==>...........................] - ETA: 3:55 - loss: 2.1558
----- Generating text after Epoch: 1
----- Generating with seed: "hould I answer thee.

[Lady Mo"
hould I answer thee.




    Thop wer whis theurGwagg

KeyboardInterrupt: ignored

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN